# Hyperparameter Grid Search for Target-Trial Emulation

This cell executes a systematic search across different propensity score trimming and weighting configurations to identify stable causal effects of daily nutritional exposures on next-night sleep architecture. The framework follows contemporary epidemiology guidelines to separate causal effects from confounding in large observational datasets by emulating a "target trial".

![Alt Text](https://media.giphy.com/media/v1.Y2lkPWVjZjA1ZTQ3dnAxMXQycXhkeW9iY2pna2lrMnk5bGlmbHFtMDd3OXFjcjRqdmlxYiZlcD12MV9naWZzX3NlYXJjaCZjdD1n/3o6MbeNr6v9XW7HNFS/giphy.gif)

### The Search Space

The grid explores two primary mechanisms used in the study to ensure the internal validity of the causal estimates: 
1. **Quantile Trimming (QUANTILE_GRID):**
   **Purpose:** To address extreme propensity scores that can lead to high variance or bias in treatment effect estimates. 
   **Mechanism:** It excludes treated observations with very low propensity scores and control observations with very high propensity scores to ensure "common support". 
2. **Weight Clipping (CLIPPING_GRID)**
   **Purpose:** To further stabilize the Hájek-type estimator by reducing the influence of extreme weights. 
   **Mechanism:** Weights are clipped at specified upper and lower percentiles to reduce the influence of extreme values.
   
Following the paper's robustness protocols, a result for a specific exposure is considered reliable only if it meets these conditions within the grid search: 
- **Covariate Balance:** Reaches an Absolute Standardized Mean Difference (ASMD) of $\le 0.10$ for general confounders. Strict Structural Balance: Reaches an ASMD of $\le 0.05$ for key structural confounders (Age, Sex, BMI) and baseline sleep characteristics.
- **Stability:** Average Treatment Effect (ATE) estimates must be stable across reasonable trimming and weighting configurations.
- **Negative Controls:** Shows no evidence of systematic treatment effects on food-insensitive outcomes like heart-rate signal quality or sleep position.

### Execution Flow

1. **Exposure Selection:** Iterates through 25 prespecified dietary exposures covering diet quality, macronutrients, micronutrients, and meal timing.
2. **Propensity Scoring:** Fits a CatBoost gradient-boosted tree classifier to estimate the probability of exposure based on a comprehensive set of baseline and time-varying covariates.
3. **Experimentation:** Evaluates combinations of trimming (q) and clipping (clip) to find a configuration that satisfies the strict balance and validity thresholds required for causal interpretation.

In [ ]:
import warnings
from dataclasses import replace

import numpy as np

from scripts.helpers import run_experiment
from scripts.propensity import get_propensity_scores
from variables.variables import *

warnings.filterwarnings("ignore")

# ================================================================
# Hyperparameter grids
# ================================================================

QUANTILE_GRID = np.array([0.0, 0.001, 0.01, 0.0125, 0.015, 0.0175, 0.025, 0.05, 0.075])
CLIPPING_PERCENTS = np.array([0.5, 1.0, 1.5, 2.5, 3.5, 5.0, 7.0, 7.5, 10.0, 12.5])
CLIPPING_GRID = [(p, 100.0 - p) for p in CLIPPING_PERCENTS]

# ================================================================
# Loop
# ================================================================

for exposure, vals in EXPOSURES.items():
    print(f"\n{exposure}")

    base_cfg = replace(
        BASE_CONFIG,
        method=vals["method"],
        limit=vals["cutoff"],
    )

    df, kwargs, X, shap_values = get_propensity_scores(
        exposure=exposure,
        config=base_cfg.__dict__,
        variables=variable_config,
        file=DATAFRAME_PATH,
    )

    passed_strict = any(
        run_experiment(
            config=replace(
                base_cfg,
                q=float(q),
                clip=clip,
            ).__dict__,
            variable_config=variable_config,
            df=df,
            kwargs=kwargs,
            X=X,
            shap_values=shap_values,
        )
        == "PASS_STRICT"
        for q in QUANTILE_GRID
        for clip in CLIPPING_GRID
    )